# Mine: layer_id as carry

In [3]:
import jax
import jax.numpy as jnp
import flax.linen as nn

batch_size = 32
feature_dim = 768
layers = 12
rng = jax.random.key(42)
x = jnp.ones((batch_size, feature_dim))

class ResidualMLPBlock(nn.Module):
  feature_dim: int = 3
  @nn.compact
  def __call__(self, layer_id, x):
    h = nn.Dense(features=self.feature_dim)(x)
    h = nn.relu(h)
    o = x+h
    return layer_id+1, o
    # out = {}
    # out['layer_id'] = layer_id
    # out['o'] = o
    # return layer_id+1, o, out

class ResidualMLP(nn.Module):
  n_layers: int = 4
  feature_dim: int = 3
  remat_policy: str = "nothing_saveable"

  @nn.compact
  def __call__(self, x):

    remat_block = nn.remat(
      ResidualMLPBlock, 
      prevent_cse=False,
      policy=getattr(jax.checkpoint_policies, self.remat_policy, None),
    )

    scan_block = nn.scan(
      remat_block, 
      variable_axes={'params': 0},
      split_rngs={'params': True},
      in_axes=nn.broadcast,
      length=self.n_layers)(
        feature_dim=self.feature_dim,)

    init_layer_id = 0
    final_layer_id, y = scan_block(init_layer_id, x)
    return final_layer_id, y
  

model = ResidualMLP(n_layers=layers, feature_dim=feature_dim)
variables = model.init(rng, x)
layer_id, y = model.apply(variables, x)
print(f"variable shapes: {jax.tree.map(lambda x: x.shape, variables)}")
print(f"layer_id: {layer_id}")
print(f"y shape: {y.shape}")

variable shapes: {'params': {'ScanCheckpointResidualMLPBlock_0': {'Dense_0': {'bias': (12, 768), 'kernel': (12, 768, 768)}}}}
layer_id: 12
y shape: (12, 32, 768)


# Mine: x and layer id as carry

In [5]:
import jax
import jax.numpy as jnp
import flax.linen as nn

batch_size = 32
feature_dim = 768
layers = 12
rng = jax.random.key(42)
x = jnp.ones((batch_size, feature_dim))

class ResidualMLPBlock(nn.Module):
  feature_dim: int = 3
  @nn.compact
  def __call__(self, carry, _):
    layer_id, x = carry
    h = nn.Dense(features=self.feature_dim)(x)
    h = nn.relu(h)
    o = x+h
    carry = layer_id+1, o
    return carry, None
    # out = {}
    # out['layer_id'] = layer_id
    # out['o'] = o
    # return layer_id+1, o, out

class ResidualMLP(nn.Module):
  n_layers: int = 4
  feature_dim: int = 3
  remat_policy: str = "nothing_saveable"

  @nn.compact
  def __call__(self, x):

    remat_block = nn.remat(
      ResidualMLPBlock, 
      prevent_cse=False,
      policy=getattr(jax.checkpoint_policies, self.remat_policy, None),
    )

    scan_block = nn.scan(
      remat_block, 
      variable_axes={'params': 0},
      split_rngs={'params': True},
      in_axes=nn.broadcast,
      length=self.n_layers)(
        feature_dim=self.feature_dim,)

    init_layer_id = 0
    carry = init_layer_id, x
    final_carry, _ = scan_block(carry, None)
    final_layer_id, y = final_carry
    return final_layer_id, y
  

model = ResidualMLP(n_layers=layers, feature_dim=feature_dim)
variables = model.init(rng, x)
layer_id, y = model.apply(variables, x)
print(f"variable shapes: {jax.tree.map(lambda x: x.shape, variables)}")
print(f"layer_id: {layer_id}")
print(f"y shape: {y.shape}")

variable shapes: {'params': {'ScanCheckpointResidualMLPBlock_0': {'Dense_0': {'bias': (12, 768), 'kernel': (12, 768, 768)}}}}
layer_id: 12
y shape: (32, 768)


# My attempt

In [12]:
import jax
import jax.numpy as jnp
import flax.linen as nn

batch_size = 32
feature_dim = 768
layers = 12
rng = jax.random.key(42)
x = jnp.ones((batch_size, feature_dim))

class ResidualMLPBlock(nn.Module):
    feature_dim: int = 3
    @nn.compact
    def __call__(self, layer_id, x):
        h = nn.Dense(features=self.feature_dim)(x)
        h = nn.relu(h)
        o = x + h
        out = {}
        out['layer_id'] = layer_id
        out['o'] = o
        return layer_id + 1, o, out

class ResidualMLP(nn.Module):
    n_layers: int = 4
    feature_dim: int = 3
    remat_policy: str = "nothing_saveable"

    @nn.compact
    def __call__(self, x):
        block = nn.remat(
            ResidualMLPBlock, 
            prevent_cse=False,
            policy=getattr(jax.checkpoint_policies, self.remat_policy, None),
        )(feature_dim=self.feature_dim)

        def scan_fn(layer_id, x):
            next_layer_id, o, out = block(layer_id, x)
            return next_layer_id, (o, out)

        ScanMLP = nn.scan(
            scan_fn,
            variable_axes={'params': 0},
            split_rngs={'params': True},
            in_axes=nn.broadcast,
            length=self.n_layers
        )

        init_layer_id = 0
        final_layer_id, (y, outs) = ScanMLP(init_layer_id, x)
        return final_layer_id, y, outs

model = ResidualMLP(n_layers=layers, feature_dim=feature_dim)
variables = model.init(rng, x)
layer_id, y, outs = model.apply(variables, x)

print(f"variable shapes: {jax.tree_map(lambda x: x.shape, variables)}")
print(f"layer_id: {layer_id}")
print(f"y shape: {y.shape}")
print(f"outs structure: {jax.tree_map(lambda x: x.shape, outs)}")

> /mnt/vlm-pd/miniconda3/envs/vlm/lib/python3.10/site-packages/flax/linen/transforms.py(420)wrapped_fn()
    418   def wrapped_fn(self, *args, **kwargs):
    419     import pdb; pdb.set_trace()
--> 420     state = self._state.export()
    421 
    422     # make a scope-function to transform



AttributeError: 'int' object has no attribute '_state'

# Big Vision

In [3]:
class ResidualMLPBlock(nn.Module):
  feature_dim: int = 3
  @nn.compact
  def __call__(self, x, deterministic=True):
    out = {}
    h = nn.Dense(features=self.feature_dim)(x)
    h = nn.relu(h)
    o = out['o'] = x+h
    return o, out

class ResidualMLP(nn.Module):
  n_layers: int = 4
  feature_dim: int = 3
  remat_policy: str = "nothing_saveable"

  @nn.compact
  def __call__(self, x, deterministic=True):
    block = nn.remat(
      ResidualMLPBlock, 
      prevent_cse=False,
      static_argnums=(2,),
      policy=getattr(jax.checkpoint_policies, self.remat_policy, None),)
    
    ScanMLP = nn.scan(
      block, 
      variable_axes={'params': 0},
      split_rngs={'params': True},
      in_axes=nn.broadcast,
      length=self.n_layers)
    
    x, scan_out = ScanMLP(
      feature_dim=self.feature_dim
    )(x,deterministic)
    return x, scan_out


model = ResidualMLP(n_layers=layers, feature_dim=feature_dim)
variables = model.init(rng, x)
y, scan_out = model.apply(variables, x, True)
print(f"variable shapes: {jax.tree.map(lambda x: x.shape, variables)}")
print(f"y shape: {y.shape}")
print(f"scan_out shapes: {jax.tree.map(lambda x: x.shape, scan_out)}")

variable shapes: {'params': {'ScanCheckpointResidualMLPBlock_0': {'Dense_0': {'bias': (12, 768), 'kernel': (12, 768, 768)}}}}
y shape: (32, 768)
scan_out shapes: {'o': (12, 32, 768)}
